In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import sqlite3
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
df=pd.read_csv('Train.csv')

In [3]:
df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating
0,e8730ce3ed5e9762038d160e23d47d79,2187,1c6af3913167e3d9d26be5a29d8df1aa,4.5 stars \n I loved this book! It was incredi...,Fri May 23 13:53:31 -0700 2014,Sun Apr 03 03:56:27 -0700 2016,Sun Mar 29 00:00:00 -0700 2015,Mon Aug 25 00:00:00 -0700 2014,0,0,4
1,26317e667d9141ad245dd2c18be52d77,47212,fa1f648451baf909683ea40bda3a06fe,I've had my dad and friend after me to read th...,Thu Jan 05 21:23:23 -0800 2017,Sun Jan 08 21:09:19 -0800 2017,Sun Jan 08 00:00:00 -0800 2017,Thu Jan 05 00:00:00 -0800 2017,1,0,4
2,bf9640b81a047ee70b4f918082492f1d,40483,ae2c43f1062ab1dae0ad51f5ea3c7c56,Anyone desiring to start an exploration of S&M...,Tue Feb 18 03:29:28 -0800 2014,Fri Feb 28 01:24:54 -0800 2014,Fri Feb 28 01:54:07 -0800 2014,Tue Feb 18 00:00:00 -0800 2014,1,1,3
3,34aa99d428ad98679c3e45d117243f55,19095025,3eb21a560e2afb02ace9e44d6fe76c8b,3.75 stars \n Mal is the best. He's crazy fun ...,Fri May 08 17:06:08 -0700 2015,Sun May 21 17:12:59 -0700 2017,Mon Sep 07 00:00:00 -0700 2015,Sun Sep 06 00:00:00 -0700 2015,1,0,4
4,6b3f929609c9d97628807d13b59b0b22,9464746,6caffed66bddb57550e777f04823fdd6,I would like to begin by saying how much I app...,Tue Feb 08 15:47:53 -0800 2011,Sat Nov 05 21:12:58 -0700 2011,Sat Nov 05 00:00:00 -0700 2011,Sun Oct 30 00:00:00 -0700 2011,0,0,5


In [4]:
import re
def clean_text(text):
    # Remove HTML tags
    clean_text = re.sub('<.*?>', '', text)
    # Remove special characters
    clean_text = re.sub('[^a-zA-Z]', ' ', clean_text)
    # Convert to lowercase
    clean_text = clean_text.lower()
    return clean_text

df['review'] = df['review_text'].apply(clean_text) 

In [6]:
#df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating,review
0,e8730ce3ed5e9762038d160e23d47d79,2187,1c6af3913167e3d9d26be5a29d8df1aa,4.5 stars \n I loved this book! It was incredi...,Fri May 23 13:53:31 -0700 2014,Sun Apr 03 03:56:27 -0700 2016,Sun Mar 29 00:00:00 -0700 2015,Mon Aug 25 00:00:00 -0700 2014,0,0,4,stars i loved this book it was incredib...
1,26317e667d9141ad245dd2c18be52d77,47212,fa1f648451baf909683ea40bda3a06fe,I've had my dad and friend after me to read th...,Thu Jan 05 21:23:23 -0800 2017,Sun Jan 08 21:09:19 -0800 2017,Sun Jan 08 00:00:00 -0800 2017,Thu Jan 05 00:00:00 -0800 2017,1,0,4,i ve had my dad and friend after me to read th...
2,bf9640b81a047ee70b4f918082492f1d,40483,ae2c43f1062ab1dae0ad51f5ea3c7c56,Anyone desiring to start an exploration of S&M...,Tue Feb 18 03:29:28 -0800 2014,Fri Feb 28 01:24:54 -0800 2014,Fri Feb 28 01:54:07 -0800 2014,Tue Feb 18 00:00:00 -0800 2014,1,1,3,anyone desiring to start an exploration of s m...
3,34aa99d428ad98679c3e45d117243f55,19095025,3eb21a560e2afb02ace9e44d6fe76c8b,3.75 stars \n Mal is the best. He's crazy fun ...,Fri May 08 17:06:08 -0700 2015,Sun May 21 17:12:59 -0700 2017,Mon Sep 07 00:00:00 -0700 2015,Sun Sep 06 00:00:00 -0700 2015,1,0,4,stars mal is the best he s crazy fun a...
4,6b3f929609c9d97628807d13b59b0b22,9464746,6caffed66bddb57550e777f04823fdd6,I would like to begin by saying how much I app...,Tue Feb 08 15:47:53 -0800 2011,Sat Nov 05 21:12:58 -0700 2011,Sat Nov 05 00:00:00 -0700 2011,Sun Oct 30 00:00:00 -0700 2011,0,0,5,i would like to begin by saying how much i app...


In [7]:
#Tokenization
from nltk.tokenize import word_tokenize
df['review'] = df['review'].apply(word_tokenize)

In [8]:
#Remove stopwords
from nltk.corpus import stopwords
default_stopwords = set(stopwords.words('english'))
stopwords_to_remove = {'not','no','nor','but'}
updated_stopwords = default_stopwords - stopwords_to_remove
def remove_stopwords(tokens):
    return [token for token in tokens if token.lower() not in updated_stopwords]

df['review'] = df['review'].apply(remove_stopwords)

In [9]:
#df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating,review
0,e8730ce3ed5e9762038d160e23d47d79,2187,1c6af3913167e3d9d26be5a29d8df1aa,4.5 stars \n I loved this book! It was incredi...,Fri May 23 13:53:31 -0700 2014,Sun Apr 03 03:56:27 -0700 2016,Sun Mar 29 00:00:00 -0700 2015,Mon Aug 25 00:00:00 -0700 2014,0,0,4,"[stars, loved, book, incredibly, intricate, we..."
1,26317e667d9141ad245dd2c18be52d77,47212,fa1f648451baf909683ea40bda3a06fe,I've had my dad and friend after me to read th...,Thu Jan 05 21:23:23 -0800 2017,Sun Jan 08 21:09:19 -0800 2017,Sun Jan 08 00:00:00 -0800 2017,Thu Jan 05 00:00:00 -0800 2017,1,0,4,"[dad, friend, read, series, years, friend, gav..."
2,bf9640b81a047ee70b4f918082492f1d,40483,ae2c43f1062ab1dae0ad51f5ea3c7c56,Anyone desiring to start an exploration of S&M...,Tue Feb 18 03:29:28 -0800 2014,Fri Feb 28 01:24:54 -0800 2014,Fri Feb 28 01:54:07 -0800 2014,Tue Feb 18 00:00:00 -0800 2014,1,1,3,"[anyone, desiring, start, exploration, erotica..."
3,34aa99d428ad98679c3e45d117243f55,19095025,3eb21a560e2afb02ace9e44d6fe76c8b,3.75 stars \n Mal is the best. He's crazy fun ...,Fri May 08 17:06:08 -0700 2015,Sun May 21 17:12:59 -0700 2017,Mon Sep 07 00:00:00 -0700 2015,Sun Sep 06 00:00:00 -0700 2015,1,0,4,"[stars, mal, best, crazy, fun, says, things, m..."
4,6b3f929609c9d97628807d13b59b0b22,9464746,6caffed66bddb57550e777f04823fdd6,I would like to begin by saying how much I app...,Tue Feb 08 15:47:53 -0800 2011,Sat Nov 05 21:12:58 -0700 2011,Sat Nov 05 00:00:00 -0700 2011,Sun Oct 30 00:00:00 -0700 2011,0,0,5,"[would, like, begin, saying, much, appreciate,..."


In [10]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

df['review'] = df['review'].apply(lemmatize)

In [11]:
destr=[]
for row in df['review'] :
    code_string=' '.join(row)
    destr.append(code_string)
df['destr_review'] = destr

In [12]:
df.to_csv('preprocess.csv', index=False)